# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [48]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest
from statsmodels.stats.proportion import proportions_ztest

from scipy.stats import ttest_ind, norm, t, ttest_1samp
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [49]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [50]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

* Hipotesis: El salario por hora medio es significativamente diferente de $30/hora.
* Hipotesis nula: El salario por hora medio no es significativamente diferente de $30/hora.

In [51]:
# Crear un dataframe quitando los valores NaN y dejando solo los salarios por hora
salarios_por_hora = salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Hourly Rate'].dropna().values

# Valor hipotético de la media de la hipotesis nula
media_hipotetica = 30

# test de hipótesis
t_statistic, p_value = ttest_1samp(salarios_por_hora, media_hipotetica)

# Mostrar los resultados
print(f"T estadística: {t_statistic}")
print(f"Valor P: {p_value}")

if p_value < 0.05:
    print("Se rechaza la hipótesis nula <> el salario por hora es significativamente diferente de $30/hora.")
else:
    print("No se rechaza la hipótesis nula <> no hay suficiente evidencia para afirmar que el salario por hora es diferente de $30/hora.")

T estadística: 20.6198057854942
Valor P: 4.3230240486229894e-92
Se rechaza la hipótesis nula <> el salario por hora es significativamente diferente de $30/hora.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [52]:
# Calcular la media de la muestra
media_muestra = np.mean(salarios_por_hora)

# Calcular el error estándar de la media (SEM)
error_estandar = sem(salarios_por_hora)

# Definir el nivel de confianza (95%)
nivel_confianza = 0.95

# Calcular los grados de libertad
grados_libertad = len(salarios_por_hora) - 1

# Calcular el intervalo de confianza
intervalo_confianza = t.interval(nivel_confianza, df=grados_libertad, loc=media_muestra, scale=error_estandar)

# Mostrar el intervalo de confianza
print(f"Intervalo de confianza del {nivel_confianza * 100}% para el salario por hora medio: {intervalo_confianza}")

Intervalo de confianza del 95.0% para el salario por hora medio: (32.52345834488425, 33.05365708767623)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de salarios por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

* H1: El número de salarios por hora en la Ciudad de Chicago es significativamente diferente del 25% del nivel de confianza del 95%.
* H0: El número de salarios por hora en la Ciudad de Chicago NO es significativamente diferente del 25% del nivel de confianza del 95%.

In [54]:
# Parámetros para la prueba de proporciones
count = len(salarios_por_hora) # Número de éxitos en la muestra (en este caso, trabajadores por hora).
nobs = len(salarios_por_hora)   # Número total de observaciones en la muestra.
value = 0.25  # Valor del 25%

# hipótesis de proporciones
z_statistic, p_value = proportions_ztest(count, nobs, value)

# Mostrar los resultados
print(f"Estadística Z: {z_statistic}")
print(f"Valor P: {p_value}")

if p_value < 0.05:
    print("Se rechaza la hipótesis nula <> la proporción de salarios por hora es significativamente diferente del 25%.")
else:
    print("No se rechaza la hipótesis nula <> no hay suficiente evidencia para afirmar que la proporción de salarios por hora es diferente del 25%.")

Estadística Z: inf
Valor P: 0.0
Se rechaza la hipótesis nula <> la proporción de salarios por hora es significativamente diferente del 25%.
